In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Check this site for the latest download link
# https://www.apache.org/dyn/closer.lua/spark
!wget -q https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
!tar xf spark-3.5.2-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,424 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,107 kB]
Get:13 https://r2u.stat.illinois.edu/ubunt

In [8]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").\
    appName("ITESO-ALS").\
    getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

file_path = "/content/drive/MyDrive/Colab Notebooks/datasets/sample_movielens_ratings.txt"
ratings_df = spark.read.format("csv") \
    .option("delimiter", "::") \
    .option("inferSchema", "true") \
    .option("header", "false") \
    .load(file_path) \
    .toDF("userId", "movieId", "rating", "timestamp")

ratings_df.show(n=10, truncate=False)

ratings_df.printSchema()
ratings_df.show(5)

+------+-------+------+----------+
|userId|movieId|rating|timestamp |
+------+-------+------+----------+
|0     |2      |3     |1424380312|
|0     |3      |1     |1424380312|
|0     |5      |2     |1424380312|
|0     |9      |4     |1424380312|
|0     |11     |1     |1424380312|
|0     |12     |2     |1424380312|
|0     |15     |1     |1424380312|
|0     |17     |1     |1424380312|
|0     |19     |1     |1424380312|
|0     |21     |1     |1424380312|
+------+-------+------+----------+
only showing top 10 rows

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: integer (nullable = true)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     0|      2|     3|1424380312|
|     0|      3|     1|1424380312|
|     0|      5|     2|1424380312|
|     0|      9|     4|1424380312|
|     0|     11|     1|1424380312|
+------+-------+------+----------+
onl

In [9]:
from pyspark.ml.recommendation import ALS

als = ALS(
    maxIter=10,
    regParam=0.1,
    userCol="userId",
    itemCol="movieId",
    ratingCol="rating",
    coldStartStrategy="drop"
)

model = als.fit(ratings_df)

In [10]:
from pyspark.ml.evaluation import RegressionEvaluator

predictions = model.transform(ratings_df)

evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
)
rmse = evaluator.evaluate(predictions)
print(f"Root-mean-square error (RMSE): {rmse}")

Root-mean-square error (RMSE): 0.397335534604917


In [11]:
user_recommendations = model.recommendForAllUsers(numItems=5)
user_recommendations.show(truncate=False)

+------+-------------------------------------------------------------------------------------+
|userId|recommendations                                                                      |
+------+-------------------------------------------------------------------------------------+
|20    |[{22, 3.669611}, {94, 3.159118}, {68, 3.1273239}, {88, 3.0766838}, {77, 3.0075834}]  |
|10    |[{92, 3.3601632}, {2, 3.1238763}, {40, 2.8678107}, {49, 2.7868533}, {12, 2.739804}]  |
|0     |[{92, 2.9841022}, {2, 2.7240503}, {62, 2.5989156}, {9, 2.3311102}, {12, 2.3006241}]  |
|1     |[{62, 3.050136}, {68, 2.996803}, {22, 2.7656898}, {77, 2.6151948}, {28, 2.3107836}]  |
|21    |[{29, 4.321138}, {52, 4.2251186}, {53, 3.9655445}, {63, 3.4733353}, {93, 3.4296377}] |
|11    |[{32, 4.8091135}, {30, 4.789024}, {27, 4.6797547}, {18, 4.671916}, {23, 4.4069214}]  |
|12    |[{46, 5.6145906}, {55, 4.583866}, {64, 4.1072335}, {17, 4.0981135}, {48, 4.017547}]  |
|22    |[{75, 4.6104155}, {74, 4.4301834}, {88, 4.